Naive bayes, XGBoost, SVM

In [10]:
import pandas as pd

import re
import string
import nltk
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import RepeatedStratifiedKFold,cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
import pickle

from sklearn import svm


Loading data

In [50]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [51]:
# train['text'] = train['location'].astype(str) +" " +train['text']
# test['text'] = test['location'].astype(str) +" " +train['text'] SHIT LOCATION

In [39]:
train = train.drop(columns=['id','keyword', 'location'], axis = 1)
# test = test.drop(columns=['id','keyword', 'location'], axis = 1)

In [72]:
# def clean(data):
#     data = data.drop(columns=['id','keyword', 'location'], axis = 1) # dropping unwanted columns
#     data.text=data.text.apply(lambda x:x.lower()) # converting data to lowercase
#     data.text=data.text.apply(lambda x:re.sub('\[.*?\]', '', x) ) #removing square brackets
#     data.text=data.text.apply(lambda x:re.sub('<.*?>+', '', x) )
#     data.text=data.text.apply(lambda x:re.sub('https?://\S+|www\.\S+', '', x) )  #removing hyperlink
#     data.text=data.text.apply(lambda x:re.sub('[%s]' % re.escape(string.punctuation), '', x) ) #removing puncuation
#     data.text=data.text.apply(lambda x:re.sub('\n' , '', x) )
#     data.text=data.text.apply(lambda x:re.sub('\w*\d\w*' , '', x) ) #remove words containing numbers
#     # converting data to string
#     for i in range(len(data)):
#         data.text[i] = str(data.text[i])
# #train.text.head()

# df = clean(train)
# #clean(test)

Preprocessing

In [52]:
train.text=train.text.apply(lambda x:x.lower() )
test.text=test.text.apply(lambda x:x.lower())

#removing square brackets
train.text=train.text.apply(lambda x:re.sub('\[.*?\]', '', x) )
test.text=test.text.apply(lambda x:re.sub('\[.*?\]', '', x) )
train.text=train.text.apply(lambda x:re.sub('<.*?>+', '', x) )
test.text=test.text.apply(lambda x:re.sub('<.*?>+', '', x) )

#removing hyperlink
train.text=train.text.apply(lambda x:re.sub('https?://\S+|www\.\S+', '', x) )
test.text=test.text.apply(lambda x:re.sub('https?://\S+|www\.\S+', '', x) )
#removing puncuation
train.text=train.text.apply(lambda x:re.sub('[%s]' % re.escape(string.punctuation), '', x) )
test.text=test.text.apply(lambda x:re.sub('[%s]' % re.escape(string.punctuation), '', x) )
train.text=train.text.apply(lambda x:re.sub('\n' , '', x) )
test.text=test.text.apply(lambda x:re.sub('\n', '', x) )
#remove words containing numbers
train.text=train.text.apply(lambda x:re.sub('\w*\d\w*' , '', x) )
test.text=test.text.apply(lambda x:re.sub('\w*\d\w*', '', x) )

train.text = train['text'].str.replace('nan', '')
test.text = test['text'].str.replace('nan', '')

train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,our deeds are the reason of this earthquake m...,1
1,4,NaN,NaN,forest fire near la ronge sask canada,1
2,5,NaN,NaN,all residents asked to shelter in place are b...,1
3,6,NaN,NaN,people receive wildfires evacuation orders i...,1
4,7,NaN,NaN,just got sent this photo from ruby alaska as ...,1


In [53]:
# train['text'] = str(train['text'])
for i in range(7613):
    train.text[i] = str(train.text[i])
    
for i in range(3263):
    test.text[i] = str(test.text[i])

<ipython-input-53-b10448f9c2c6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.text[i] = str(train.text[i])
<ipython-input-53-b10448f9c2c6>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.text[i] = str(test.text[i])


In [54]:
token=nltk.tokenize.RegexpTokenizer(r'\w+')
#applying token
train.text=train.text.apply(lambda x:token.tokenize(x))
test.text=test.text.apply(lambda x:token.tokenize(x))
#view
display(train.text.head())

0    [our, deeds, are, the, reason, of, this, earth...
1        [forest, fire, near, la, ronge, sask, canada]
2    [all, residents, asked, to, shelter, in, place...
3    [people, receive, wildfires, evacuation, order...
4    [just, got, sent, this, photo, from, ruby, ala...
Name: text, dtype: object

In [55]:
nltk.download('stopwords')

#removing stop words
train.text=train.text.apply(lambda x:[w for w in x if w not in stopwords.words('english')])
test.text=test.text.apply(lambda x:[w for w in x if w not in stopwords.words('english')])

#view
train.text.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0    [deeds, reason, earthquake, may, allah, forgiv...
1        [forest, fire, near, la, ronge, sask, canada]
2    [residents, asked, shelter, place, notified, o...
3    [people, receive, wildfires, evacuation, order...
4    [got, sent, photo, ruby, alaska, smoke, wildfi...
Name: text, dtype: object

In [56]:
stemmer = nltk.stem.PorterStemmer()
train.text=train.text.apply(lambda x:" ".join(stemmer.stem(token) for token in x))
test.text=test.text.apply(lambda x:" ".join(stemmer.stem(token) for token in x))

#View
train.text.head()

0            deed reason earthquak may allah forgiv us
1                 forest fire near la rong sask canada
2    resid ask shelter place notifi offic evacu she...
3          peopl receiv wildfir evacu order california
4    got sent photo rubi alaska smoke wildfir pour ...
Name: text, dtype: object

In [57]:
X_train_vectors= [nltk.word_tokenize(i) for i in train.text]  
X_test_vectors= [nltk.word_tokenize(i) for i in test.text]

In [23]:
#X_train = train.text
#X_test = test.text

In [58]:
from sklearn.feature_extraction.text import TfidfVectorizer 
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
X_train_vectors_tfidf = tfidf_vectorizer.fit_transform(train['text']) 
X_test_vectors_tfidf = tfidf_vectorizer.transform(test['text'])

In [25]:
print(X_train_vectors_tfidf) ## transform() = returns document_term matrix. eg (0,12426) = 0th document word 12426. weightage = 0.26474..

  (0, 12426)	0.2647497827712146
  (0, 4351)	0.460736130793121
  (0, 312)	0.4170447253945454
  (0, 7234)	0.2954351691411343
  (0, 3506)	0.3268540017980149
  (0, 9515)	0.35041026506244183
  (0, 2932)	0.47623590352577255
  (1, 1739)	0.3882998873473426
  (1, 10152)	0.49182490935714945
  (1, 9918)	0.49182490935714945
  (1, 6502)	0.35142353046193325
  (1, 7928)	0.3145471735765239
  (1, 4190)	0.22018178245170175
  (1, 4343)	0.3074431249301026
  (2, 3908)	0.23228019043761372
  (2, 8451)	0.2273094843381881
  (2, 3843)	0.1845362036780636
  (2, 8286)	0.22212503519906254
  (2, 8152)	0.3349256403794359
  (2, 8909)	0.4624599822086023
  (2, 10461)	0.5902472841741969
  (2, 666)	0.24063191047665963
  (2, 9700)	0.28043390876899704
  (3, 1706)	0.3530488267262339
  (3, 12927)	0.37770608758404556
  :	:
  (7609, 1706)	0.24135737996572423
  (7609, 4190)	0.1936358840523296
  (7610, 5144)	0.7555893307430575
  (7610, 12644)	0.6550456192260646
  (7611, 8100)	0.29846856454890786
  (7611, 9035)	0.29846856454890786

In [59]:
count_vectorizer = CountVectorizer() 
train_vectors_count = count_vectorizer.fit_transform(train['text'])
test_vectors_count = count_vectorizer.transform(test["text"])

In [27]:
train_vectors_count

<7613x13407 sparse matrix of type '<class 'numpy.int64'>'
	with 65639 stored elements in Compressed Sparse Row format>

In [28]:
print(train_vectors_count) # (0,2932) = 0th document, word 2932 ocuuring 1 time in the document. (one document can have many lines. like each row of text column)

  (0, 2932)	1
  (0, 9515)	1
  (0, 3506)	1
  (0, 7234)	1
  (0, 312)	1
  (0, 4351)	1
  (0, 12426)	1
  (1, 4343)	1
  (1, 4190)	1
  (1, 7928)	1
  (1, 6502)	1
  (1, 9918)	1
  (1, 10152)	1
  (1, 1739)	1
  (2, 9700)	1
  (2, 666)	1
  (2, 10461)	2
  (2, 8909)	2
  (2, 8152)	1
  (2, 8286)	1
  (2, 3843)	1
  (2, 8451)	1
  (2, 3908)	1
  (3, 3843)	1
  (3, 8451)	1
  :	:
  (7609, 592)	1
  (7609, 11730)	1
  (7610, 12644)	1
  (7610, 5144)	1
  (7611, 1788)	1
  (7611, 8989)	1
  (7611, 9796)	1
  (7611, 5779)	1
  (7611, 6810)	1
  (7611, 10367)	1
  (7611, 5868)	1
  (7611, 11785)	1
  (7611, 11270)	1
  (7611, 2292)	1
  (7611, 3528)	2
  (7611, 9035)	1
  (7611, 8100)	1
  (7612, 12927)	1
  (7612, 1706)	1
  (7612, 5374)	1
  (7612, 7987)	1
  (7612, 6575)	1
  (7612, 8127)	1
  (7612, 21)	1
  (7612, 9471)	1


In [149]:
NB_Vec = MultinomialNB()
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
scores = cross_val_score(NB_Vec, train_vectors_count, train["target"], cv=cv, scoring="f1")
scores

array([0.77127244, 0.75604053, 0.74760383, 0.74822415, 0.76228209,
       0.75396825, 0.75545171, 0.75917065, 0.75851148, 0.75409836,
       0.74584323, 0.74451411, 0.76682316, 0.75911252, 0.75862069])

In [ ]:
pipe = Pipeline([('Vectors', CountVectorizer(stop_words='english')),('tfidf', TfidfTransformer()),
     ('NB', MultinomialNB())])
pipe.fit(train.text.values, train["target"].values)

In [ ]:
pred=NB_Vec.predict(test_vectors_count)

XGBoost

In [11]:
!pip install xgboost

You should consider upgrading via the 'c:\programdata\anaconda3\python.exe -m pip install --upgrade pip' command.


In [60]:
from xgboost import XGBRegressor

xgb = XGBRegressor(n_estimators=120, leanring_rate=0.075)

In [61]:
xgb.fit(train_vectors_count, train['target'])

[00:02:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "leanring_rate" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             leanring_rate=0.075, learning_rate=0.300000012, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=120, n_jobs=8,
             num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1,
             scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [62]:
predis = xgb.predict(test_vectors_count)

In [63]:
a= [1 if i>=0.5 else 0 for i in predis]  

In [64]:
test['target'] = a

In [34]:
test

,id,keyword,location,text,target
0,0,NaN,NaN,happen terribl car crash,1
1,2,NaN,NaN,heard earthquak differ citi stay safe everyon,1
2,3,NaN,NaN,forest fire spot pond gees flee across street ...,1
3,9,NaN,NaN,apocalyps light spokan wildfir,1
4,11,NaN,NaN,typhoon soudelor kill china taiwan,1
...,...,...,...,...,...
3258,10861,NaN,NaN,earthquak safeti lo angel ûò safeti fasten xrwn,1
3259,10865,NaN,NaN,storm ri wors last hurrican hardest hit yard l...,1
3260,10868,NaN,NaN,green line derail chicago,1
3261,10874,NaN,NaN,meg issu hazard weather outlook hwo,1


In [65]:
test = test.drop(columns=['keyword','location','text']) 

In [66]:
test.to_csv(r'test_1.csv', index=False)

SVM

In [170]:
svm_model = svm.SVC(kernel = 'linear', gamma='auto')
svm_model.fit(train_vectors_count, train['target'])


SVC(gamma='auto', kernel='linear')

In [171]:
pred1 = svm_model.predict(test_vectors_count)

In [172]:
pred1

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [173]:
test['target'] = pred1

In [168]:
test

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [174]:
test.to_csv(r'test_svm.csv', index=False)

In [176]:
# DT
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

In [178]:
dtree=DecisionTreeClassifier()
predictions_DT=dtree.fit(train_vectors_count, train['target']).predict(test_vectors_count)
# print('The accuracy of the Decision Tree is',round(accuracy_score(predictions_DT,y_test)*100,2))
# sns.heatmap(confusion_matrix(y_test,predictions_DT),annot=True,fmt='3.0f')
# plt.title('Confusion_matrix', y=1.05, size=15)

In [179]:
predictions_DT

array([0, 1, 1, ..., 0, 1, 0], dtype=int64)

In [180]:
test['target'] = predictions_DT

In [181]:
test.to_csv(r'test_dt.csv', index=False)